In [1]:
!nvidia-smi

Sat Feb 22 03:20:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
samples = list()
while True:
    samples.append([1])

# 순서

학습 데이터 텍스트 파일 준비

전처리

토크나이징

텐서 변환

데이터 셋 분할

데이터로더에 담기

모델링

훈련코드 작성 (파인튜닝)

테스트코드 작성

# 학습 데이터 준비

In [ ]:
import pandas as pd
df_data = pd.read_csv('/content/drive/My Drive/tabditor/FromZero/review_finance/Copy of 30000.csv', header=None)
df_data

# 전처리

In [0]:
import re
re_pat = re.compile('[^가-힇a-zA-Z0-9\s]')

In [5]:
re_pat.sub('', 'This is 테스트 문자열@#$%@%&*%^#423451. ')

'This is 테스트 문자열423451 '

In [ ]:
df_data[0].apply(lambda x: '[CLS]'+x)

In [ ]:
df_data.apply(lambda x: '[CLS]'+x[0], axis=1)

In [0]:
def my_function(x):
    return '[CLS]'+x

In [10]:
my_function('안녕하세요')

'[CLS]안녕하세요'

In [ ]:
df_data[0].apply(my_function)

In [12]:
a = lambda x='안녕하세요': '[CLS]'+x
a('안녕')

'[CLS]안녕'

In [0]:
df_data[0] = df_data[0].apply(lambda x: re_pat.sub('', x))

In [ ]:
df_data[0]

# 토크나이징

In [6]:
pip install transformers

In [7]:
import transformers
from transformers import BertTokenizer

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [18]:
tokenizer.tokenize('테스트 문장 입니다')

['테', '##스트', '문', '##장', '입', '##니다']

In [ ]:
df_data[0] = df_data[0].apply(lambda x: tokenizer.tokenize(x))
df_data[0]

In [20]:
tokenizer_un = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [22]:
df_data[0].apply(lambda x: tokenizer_un.tokenize(x))

TypeError: ignored

# tokens to ids

In [15]:
tokenizer.texts_to_sequences

AttributeError: ignored

In [0]:
dir(tokenizer)

In [24]:
import numpy as np
tokenizer.convert_tokens_to_ids(['근'], ['하'], ['신'])

TypeError: ignored

In [0]:
tokenizer.convert_ids_to_tokens([100])

In [10]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in df_data[0]]
input_ids

[[185,
  10410,
  9272,
  12692,
  9834,
  79940,
  23811,
  12605,
  47058,
  9609,
  119185,
  24982,
  48549,
  9309,
  37388,
  11287,
  9074,
  9654,
  9356,
  58303,
  48345,
  9477,
  23811,
  18392,
  12674,
  12674,
  12674,
  62885,
  9069,
  12692],
 [62885,
  8888,
  118617,
  108578,
  70915,
  47058,
  9322,
  119185,
  24982,
  48549,
  119228,
  41521,
  21614,
  9429,
  16758,
  58931,
  10622,
  9256,
  16985,
  9609,
  32158,
  11882,
  8932,
  119024,
  10739,
  9008,
  18622,
  14040,
  29669,
  8932,
  14279,
  33188,
  48345,
  21611,
  103155,
  35506,
  25387,
  25701,
  9448,
  89292,
  9638,
  35866,
  14040,
  11664,
  9966,
  70915,
  11882,
  8865,
  47181,
  9609,
  32158,
  10739,
  9008,
  72087,
  11102,
  9960,
  11287,
  9098,
  14040,
  29669,
  9318,
  118853,
  48345,
  12674,
  12674,
  12674,
  9730,
  13890,
  62885,
  119153,
  67527,
  12674,
  12674,
  12674,
  10892,
  25549,
  12674,
  12674,
  12674,
  119153,
  67527],
 [62885,
  8888,
 

In [83]:
input_ids[0][0].type()

AttributeError: ignored

# 패딩

In [0]:
# from tensorflow.keras.preprocessing.sequences import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [28]:
dir([])

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [97]:
max_len = max([len(x) for x in input_ids])
max_len

1039

In [12]:
sequences = list()

for sequence in input_ids:
    try:
        sequences.append(pad_sequences([sequence], maxlen=max_len), padding='post')
    except:
        print('error sequence: ', sequence)
        break

sequences

error sequence:  [185, 10410, 9272, 12692, 9834, 79940, 23811, 12605, 47058, 9609, 119185, 24982, 48549, 9309, 37388, 11287, 9074, 9654, 9356, 58303, 48345, 9477, 23811, 18392, 12674, 12674, 12674, 62885, 9069, 12692]


[]

In [31]:
pad_sequences([[1, 2, 3, 4, 5, 6, 7]], maxlen=5, truncating='pre')

array([[3, 4, 5, 6, 7]], dtype=int32)

In [84]:
# 이전 방법
input_ids = [pad_sequences(x, dtype='long', maxlen=max_len, padding='post') for x in input_ids]
input_ids

[array([[  185, 10410,  9272, ...,     0,     0,     0]]),
 array([[ 62885,   8888, 118617, ...,      0,      0,      0]]),
 array([[ 62885,   8888, 118617, ...,      0,      0,      0]]),
 array([[ 8877, 78123,  9405, ...,     0,     0,     0]]),
 array([[8926, 9952, 9487, ...,    0,    0,    0]]),
 array([[ 8959, 10892, 25805, ...,     0,     0,     0]]),
 array([[  9246,  32158, 118691, ...,      0,      0,      0]]),
 array([[ 9429, 41919,  9519, ...,     0,     0,     0]]),
 array([[  9625, 118920,  11018, ...,      0,      0,      0]]),
 array([[ 9651, 18778, 18227, ...,     0,     0,     0]]),
 array([[ 9703, 41521, 21614, ...,     0,     0,     0]]),
 array([[  185, 10410,  9960, ...,     0,     0,     0]]),
 array([[ 62885, 119198,  12674, ...,      0,      0,      0]]),
 array([[8926, 9952, 9487, ...,    0,    0,    0]]),
 array([[ 9757, 70915,  9410, ...,     0,     0,     0]]),
 array([[ 9426, 12674, 12674, ...,     0,     0,     0]]),
 array([[8926, 9952, 9487, ...,    0, 

In [13]:
# 새로운 방법
max_len = 512
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype='long', padding='post', truncating='post')
input_ids

array([[   185,  10410,   9272, ...,      0,      0,      0],
       [ 62885,   8888, 118617, ...,      0,      0,      0],
       [ 62885,   8888, 118617, ...,      0,      0,      0],
       ...,
       [  9964, 106249,  42769, ...,      0,      0,      0],
       [  9964, 106249,  42769, ...,  62885,  12674,  12674],
       [  9964, 106249,  42769, ...,  11903,  14867,   9659]])

In [70]:
input_ids[0].type()

AttributeError: ignored

# 데이터셋 분할

In [0]:
from sklearn.model_selection import train_test_split

In [15]:
labels = df_data[1].to_list()
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2018,
                                                                                    test_size=0.1)

In [36]:
dir(df_data[0])

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__p

In [37]:
train_labels

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


# 어텐션 마스킹

In [17]:
attention_masks = list()

for id in input_ids:

    ls_id = list()
    # print(id)
    for i in id:
        if i != 0:
            ls_id.append(1)
        else:
            ls_id.append(0)
    attention_masks.append(ls_id)

print(attention_masks)
print(len(attention_masks))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 마스킹 분할

In [0]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                         [0 for _ in range(30344)],
                                                        random_state=2018,
                                                        test_size=0.1)

# 텐서 변환

In [0]:
#train_inputs, validation_inputs, train_labels, validation_labels, train_inputs, validation_inputs


In [0]:
import torch

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)


In [20]:
train_inputs.type()

'torch.LongTensor'

# 모델링

In [0]:
from transformers import BertForSequenceClassification

In [0]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

In [23]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [0]:
model_plus = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=10)

In [0]:
model_plus

# 데이터 로더에 담기

In [0]:
#train_inputs, validation_inputs, train_labels, validation_labels


In [0]:
from torch.utils.data import TensorDataset, RandomSampler, DataLoader, SequentialSampler

In [0]:
batch_size = 2

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [32]:
!nvidia-smi

Sat Feb 22 03:19:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    23W /  75W |   7605MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [27]:
model.to('cuda')


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [0]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [34]:
for i in range(2):
    for idx, batch in enumerate(validation_dataloader):
        batch = tuple(t.to('cuda') for t in batch)
        b_input_ids, b_input_masks, b_input_labels = batch

        # print(dir(b_input_ids))
        #b_input_ids = b_input_ids.long()
        # print(b_input_ids.type())
        # break
        print(b_input_ids.shape)
        print()
        #break
        prediction = model(b_input_ids, b_input_masks)

        loss = F.cross_entropy(prediction[0], b_input_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(idx+1, '번 데이터 완료', loss)
    print(i+1, '번째 epoch 완료')
    #break

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

1371 번 데이터 완료 tensor(0.0087, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1372 번 데이터 완료 tensor(2.2999, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1373 번 데이터 완료 tensor(0.0188, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1374 번 데이터 완료 tensor(0.0317, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1375 번 데이터 완료 tensor(0.0493, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1376 번 데이터 완료 tensor(0.0698, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1377 번 데이터 완료 tensor(0.0894, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1378 번 데이터 완료 tensor(0.1030, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1379 번 데이터 완료 tensor(0.1071, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1380 번 데이터 완료 tensor(0.1011, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512])

1381 번 데이터 완료

In [30]:
prediction

(tensor([[-0.2944,  0.0018],
         [-0.2965,  0.0055]], device='cuda:0', grad_fn=<AddmmBackward>),)

In [31]:
b_input_labels

tensor([0, 0], device='cuda:0')

In [0]:
['T', '__abs__', '__add__', '__and__', '__array__', '__array_priority__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__idiv__', '__ilshift__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pow__', '__radd__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rfloordiv__', '__rmul__', '__rpow__', '__rshift__', '__rsub__', '__rtruediv__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '__xor__', '_backward_hooks', '_base', '_cdata', '_coalesced_', '_dimI', '_dimV', '_grad', '_grad_fn', '_indices', '_is_view', '_make_subclass', '_nnz', '_update_names', '_values', '_version', 'abs', 'abs_', 'acos', 'acos_', 'add', 'add_', 'addbmm', 'addbmm_', 'addcdiv', 'addcdiv_', 'addcmul', 'addcmul_', 'addmm', 'addmm_', 'addmv', 'addmv_', 'addr', 'addr_', 'align_as', 'align_to', 'all', 'allclose', 'angle', 'any', 'apply_', 'argmax', 'argmin', 'argsort', 'as_strided', 'as_strided_', 'asin', 'asin_', 'atan', 'atan2', 'atan2_', 'atan_', 'backward', 'baddbmm', 'baddbmm_', 'bernoulli', 'bernoulli_', 'bfloat16', 'bincount', 'bitwise_not', 'bitwise_not_', 'bitwise_xor', 'bitwise_xor_', 'bmm', 'bool', 'byte', 'cauchy_', 'ceil', 'ceil_', 'char', 'cholesky', 'cholesky_inverse', 'cholesky_solve', 'chunk', 'clamp', 'clamp_', 'clamp_max', 'clamp_max_', 'clamp_min', 'clamp_min_', 'clone', 'coalesce', 'conj', 'contiguous', 'copy_', 'cos', 'cos_', 'cosh', 'cosh_', 'cpu', 'cross', 'cuda', 'cumprod', 'cumsum', 'data', 'data_ptr', 'dense_dim', 'dequantize', 'det', 'detach', 'detach_', 'device', 'diag', 'diag_embed', 'diagflat', 'diagonal', 'digamma', 'digamma_', 'dim', 'dist', 'div', 'div_', 'dot', 'double', 'dtype', 'eig', 'element_size', 'eq', 'eq_', 'equal', 'erf', 'erf_', 'erfc', 'erfc_', 'erfinv', 'erfinv_', 'exp', 'exp_', 'expand', 'expand_as', 'expm1', 'expm1_', 'exponential_', 'fft', 'fill_', 'fill_diagonal_', 'flatten', 'flip', 'float', 'floor', 'floor_', 'fmod', 'fmod_', 'frac', 'frac_', 'gather', 'ge', 'ge_', 'geometric_', 'geqrf', 'ger', 'get_device', 'grad', 'grad_fn', 'gt', 'gt_', 'half', 'hardshrink', 'has_names', 'histc', 'ifft', 'imag', 'index_add', 'index_add_', 'index_copy', 'index_copy_', 'index_fill', 'index_fill_', 'index_put', 'index_put_', 'index_select', 'indices', 'int', 'int_repr', 'inverse', 'irfft', 'is_coalesced', 'is_complex', 'is_contiguous', 'is_cuda', 'is_distributed', 'is_floating_point', 'is_leaf', 'is_mkldnn', 'is_nonzero', 'is_pinned', 'is_quantized', 'is_same_size', 'is_set_to', 'is_shared', 'is_signed', 'is_sparse', 'isclose', 'item', 'kthvalue', 'layout', 'le', 'le_', 'lerp', 'lerp_', 'lgamma', 'lgamma_', 'log', 'log10', 'log10_', 'log1p', 'log1p_', 'log2', 'log2_', 'log_', 'log_normal_', 'log_softmax', 'logdet', 'logical_not', 'logical_not_', 'logical_xor', 'logical_xor_', 'logsumexp', 'long', 'lstsq', 'lt', 'lt_', 'lu', 'lu_solve', 'map2_', 'map_', 'masked_fill', 'masked_fill_', 'masked_scatter', 'masked_scatter_', 'masked_select', 'matmul', 'matrix_power', 'max', 'mean', 'median', 'min', 'mm', 'mode', 'mul', 'mul_', 'multinomial', 'mv', 'mvlgamma', 'mvlgamma_', 'name', 'names', 'narrow', 'narrow_copy', 'ndim', 'ndimension', 'ne', 'ne_', 'neg', 'neg_', 'nelement', 'new', 'new_empty', 'new_full', 'new_ones', 'new_tensor', 'new_zeros', 'nonzero', 'norm', 'normal_', 'numel', 'numpy', 'orgqr', 'ormqr', 'output_nr', 'permute', 'pin_memory', 'pinverse', 'polygamma', 'polygamma_', 'pow', 'pow_', 'prelu', 'prod', 'put_', 'q_per_channel_axis', 'q_per_channel_scales', 'q_per_channel_zero_points', 'q_scale', 'q_zero_point', 'qr', 'qscheme', 'random_', 'real', 'reciprocal', 'reciprocal_', 'record_stream', 'refine_names', 'register_hook', 'reinforce', 'relu', 'relu_', 'remainder', 'remainder_', 'rename', 'rename_', 'renorm', 'renorm_', 'repeat', 'repeat_interleave', 'requires_grad', 'requires_grad_', 'reshape', 'reshape_as', 'resize', 'resize_', 'resize_as', 'resize_as_', 'retain_grad', 'rfft', 'roll', 'rot90', 'round', 'round_', 'rsqrt', 'rsqrt_', 'scatter', 'scatter_', 'scatter_add', 'scatter_add_', 'select', 'set_', 'shape', 'share_memory_', 'short', 'sigmoid', 'sigmoid_', 'sign', 'sign_', 'sin', 'sin_', 'sinh', 'sinh_', 'size', 'slogdet', 'smm', 'softmax', 'solve', 'sort', 'sparse_dim', 'sparse_mask', 'sparse_resize_', 'sparse_resize_and_clear_', 'split', 'split_with_sizes', 'sqrt', 'sqrt_', 'squeeze', 'squeeze_', 'sspaddmm', 'std', 'stft', 'storage', 'storage_offset', 'storage_type', 'stride', 'sub', 'sub_', 'sum', 'sum_to_size', 'svd', 'symeig', 't', 't_', 'take', 'tan', 'tan_', 'tanh', 'tanh_', 'to', 'to_dense', 'to_mkldnn', 'to_sparse', 'tolist', 'topk', 'trace', 'transpose', 'transpose_', 'triangular_solve', 'tril', 'tril_', 'triu', 'triu_', 'trunc', 'trunc_', 'type', 'type_as', 'unbind', 'unflatten', 'unfold', 'uniform_', 'unique', 'unique_consecutive', 'unsqueeze', 'unsqueeze_', 'values', 'var', 'view', 'view_as', 'where', 'zero_']


In [0]:
train_inputs, validation_inputs, train_labels, validation_labels